##IMAGE

In [ ]:
!pip install --pre timm

In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import matplotlib.pyplot as plt
from PIL import Image

from torch.utils.data import Dataset,ConcatDataset,DataLoader
from torch import nn, optim
import torch
from sklearn.model_selection import train_test_split

from torchvision import transforms
import timm 
import timm.optim
from timm.data.transforms_factory import create_transform
from torch.optim import lr_scheduler
import os
from fastai.vision.all import *
from timm import create_model

In [ ]:
train_df = pd.read_csv("/kaggle/input/hackathon-online-find-the-buildings/train.csv")
train_df

In [ ]:
# train_df
for i in train_df['file']:
  train_df.loc[train_df['file'] == i, 'file'] = 'kaggle/input/hackathon-online-find-the-buildings/train/train/'+i

In [ ]:
import matplotlib.pyplot as plt

# Count the values in the 'fruit' column
counts = train_df['label'].value_counts()

# Display the counts
print(counts)

# Plot the counts as a bar chart
counts.plot(kind='bar')
plt.show()

In [ ]:
set_seed(69420, reproducible=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
root_dir = '/kaggle/input/hackathon-online-find-the-buildings/test/'

In [ ]:
test_dir = '/kaggle/input/hackathon-online-find-the-buildings/test/'
data = []

for i in os.listdir(root_dir):
    file_path = os.path.join(root_dir, i).lstrip('/')
    data.append({'file': file_path})
    
test_df = pd.DataFrame(data)

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
image_size = 448

In [ ]:
dls = ImageDataLoaders.from_df(train_df, path = '/', valid_pct=0.1, seed=123, shuffle_train = True,
fn_col='file', label_col='label', bs=16, num_workers=16, item_tfms=Resize(image_size),
batch_tfms=[*aug_transforms(max_rotate=5, max_zoom=1.1, max_lighting=0.2, max_warp=0.2),
Normalize.from_stats(*imagenet_stats)], balance=True)

In [ ]:
dls.show_batch()

In [ ]:
timm.list_models("*eva**", pretrained =True)

In [ ]:
from fastai.metrics import F1Score

f1 = F1Score(average='macro')

In [ ]:
# loss_func2 = CrossEntropyLossFlat(weight=class_weights)
save_cb = SaveModelCallback(monitor='valid_loss',at_end=False,every_epoch=True)
early_stop_cb = EarlyStoppingCallback(monitor='valid_loss',min_delta=0.001 ,patience=10)

# Create a list of callbacks
callbacks = [save_cb,early_stop_cb]   #

In [ ]:
# model = create_model("eva02_large_patch14_448.mim_in22k_ft_in22k_in1k", pretrained=True, num_classes=2)

In [ ]:
model = create_model("eva02_base_patch14_448.mim_in22k_ft_in22k_in1k", pretrained=True, num_classes=2)

In [ ]:
learn = vision_learner(dls, "eva02_base_patch14_448.mim_in22k_ft_in22k_in1k",pretrained=True, path='/kaggle/working/', metrics=[error_rate,f1, accuracy],
                       cbs=[ShowGraphCallback()] ).to_fp16()

In [ ]:
learn.model = torch.nn.DataParallel(learn.model)

In [ ]:
learn.show_results()

In [ ]:
learn.lr_find(suggest_funcs=(valley, slide))

In [ ]:
save_callback = SaveModelCallback(monitor='accuracy', fname='best_model')

In [ ]:
learn.fine_tune(10,freeze_epochs=3,cbs=callbacks)

In [ ]:
learn.load('/kaggle/working/models/model_6')

In [ ]:
learn.validate()

In [ ]:
learn.export('/kaggle/working/eva02_base_patch14_448.mim_in22k_ft_in22k_in1k_augmented_val_005.pkl')

In [ ]:
tst_dl=dls.test_dl(test_df)

In [ ]:
probs,_ ,idx=learn.get_preds(dl=tst_dl,with_decoded=True)
idx

In [ ]:
mapping=dict(enumerate(dls.vocab))
results=pd.Series(idx.numpy(), name='predict').map(mapping)
results

In [ ]:
submission = pd.concat([test_df,results],axis=1)

In [ ]:
submission

In [ ]:
get_filename = lambda x: x.split('/')[-1]
submission['file'] = submission['file'].apply(get_filename)

In [ ]:
submission = submission[['file', 'predict']]
submission.rename(columns = {'predict':'label'}, inplace = True)
submission

In [ ]:
submission['label'].value_counts()

In [ ]:
submission.to_csv('/kaggle/working/eva_02_find_building_no_augment.csv',index=False)

In [9]:
import pandas as pd
import numpy as np

model1_preds = pd.read_csv("/kaggle/input/best-result-for-find-the-building/eva02_base_patch14_448.mim_in22k_ft_in22k_in1k_find_building.csv")
model2_preds = pd.read_csv("/kaggle/input/best-result-for-find-the-building/eva_02_find_building (1).csv")
model3_preds = pd.read_csv("/kaggle/input/best-result-for-find-the-building/eva_02_find_building_no_augment.csv")

merged_preds = pd.merge(model1_preds, model2_preds, on="file")
merged_preds = pd.merge(merged_preds, model3_preds, on="file")

def majority_voting(row):
    counts = np.bincount(row)
    return np.argmax(counts)

merged_preds["Ensemble_Prediction"] = merged_preds.iloc[:, 1:].apply(majority_voting, axis=1)
merged_preds = merged_preds.drop(['label_x', 'label_y', 'label'], axis = 1)
merged_preds = merged_preds.rename(columns={"Ensemble_Prediction": "label"})
merged_preds.to_csv("ensemble_preds.csv", index=False)